In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import sys
import os

project_path = '/content/drive/My Drive/ICON'
src_path = os.path.join(project_path, 'src')

if src_path not in sys.path:
    sys.path.append(src_path)
try:
    os.chdir(project_path)
except FileNotFoundError:
    print(f"cartella non trovata")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from loader_dati import carica_e_pulisci_dati
from eda import esegui_eda_base, traccia_distribuzioni
from preprocessing import preprocessa_dati
from clustering import trova_k_ottimale, applica_kmeans_e_aggiungi_feature
from modelli import ottieni_modelli, ottieni_griglie_parametri
from valutazioni import ottimizza_e_valuta_modelli_cv, stampa_risultati_cv

warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

In [ ]:
PERCORSO_FILE_CSV = 'dataset/heart-disease/heart_disease_uci.csv'
COLONNA_TARGET = 'target'
NUM_SPLIT_ESTERNI = 10
NUM_SPLIT_INTERNI = 5
MAX_K_CLUSTERING = 8
STATO_CASUALE = 42
K_OTTIMALE = 4

In [ ]:

df_dati = carica_e_pulisci_dati(PERCORSO_FILE_CSV)
if df_dati is not None:
    print("\nprime righe dati puliti:")
    print(df_dati.head())
    print("\ninfo dati puliti:")
    df_dati.info()
else:
    print("loead e pulizia falliti.")

In [ ]:
if df_dati is not None:
    esegui_eda_base(df_dati)
    traccia_distribuzioni(df_dati)
else:
    print("load fallito")

In [ ]:
 #preprocessing Iniziale (no cluster)
X_proc_no_cluster, y_target, preprocessore_no_cluster, nomi_feature_no_cluster = None, None, None, None
if df_dati is not None:
    X_proc_no_cluster, y_target, preprocessore_no_cluster, nomi_feature_no_cluster = preprocessa_dati(df_dati.copy(), COLONNA_TARGET)
else:
    print("errore caricamento, preprocessing saltato")


if X_proc_no_cluster is not None:
    try:
        print("\nprime righe dati processati (senza cluster):")
        print(pd.DataFrame(X_proc_no_cluster, columns=nomi_feature_no_cluster).head())
    except ValueError:
         print("impossibile mostrare dataframe preprocessato.")

In [ ]:
# Per trovare il K ottimale
if X_proc_no_cluster is not None:
    trova_k_ottimale(X_proc_no_cluster, max_k=MAX_K_CLUSTERING, stato_casuale=STATO_CASUALE)
else:
    print("clustering saltato errore preprocessing.")

In [ ]:
# applicazione k-means e feature con cluster
X_proc_con_cluster, preprocessore_con_cluster, nomi_feature_con_cluster = None, None, None
if X_proc_no_cluster is not None and df_dati is not None:
    X_proc_con_cluster, preprocessore_con_cluster, nomi_feature_con_cluster = applica_kmeans_e_aggiungi_feature(
        df_dati.drop(COLONNA_TARGET, axis=1),
        X_proc_no_cluster,
        K_OTTIMALE,
        stato_casuale=STATO_CASUALE
    )
else:
    print("feature cluster saltata.")

if X_proc_con_cluster is not None:
    try:
        print("\nrighe dati processati (cluster):")
        print(pd.DataFrame(X_proc_con_cluster, columns=nomi_feature_con_cluster).head())
    except ValueError:
        print("impossibile mostrare dataframe con cluster")

In [ ]:
#modelli e grid
tutti_i_modelli = ottieni_modelli(stato_casuale=STATO_CASUALE)
griglie_parametri = ottieni_griglie_parametri()

modello_nb = {'Naive Bayes': tutti_i_modelli['Naive Bayes']}
modelli_standard = {nome: modello for nome, modello in tutti_i_modelli.items() if nome != 'Naive Bayes'}
griglie_standard = {nome: griglia for nome, griglia in griglie_parametri.items() if nome in modelli_standard}

print("Modelli e griglie definiti.")
print("Modelli Standard:", list(modelli_standard.keys()))
print("Griglie Standard:", griglie_standard)

In [ ]:
#valutazione Naive Bayes senza cluster
print("\nvalutazione NB")
risultati_nb = None
if X_proc_no_cluster is not None and y_target is not None:
    risultati_nb = ottimizza_e_valuta_modelli_cv(
        modello_nb, {}, X_proc_no_cluster, y_target,
        num_split_esterni=NUM_SPLIT_ESTERNI, num_split_interni=NUM_SPLIT_INTERNI, stato_casuale=STATO_CASUALE
    )
else:
    print("valutazione NB saltata.")

In [ ]:
#valutazione modelli normali senza cluster
print("valutazione modelli normali senza cluster")
risultati_no_cluster = None
if X_proc_no_cluster is not None and y_target is not None:
    risultati_no_cluster = ottimizza_e_valuta_modelli_cv(
        modelli_standard, griglie_standard, X_proc_no_cluster, y_target,
        num_split_esterni=NUM_SPLIT_ESTERNI, num_split_interni=NUM_SPLIT_INTERNI, stato_casuale=STATO_CASUALE
    )
else:
    print("valutazione modelli normali senza cluster")

In [ ]:
#valutazione modelli normali con cluster
print("valutazione modelli normali con cluster")
risultati_con_cluster = None
if X_proc_con_cluster is not None and y_target is not None:

    modelli_standard_clust = ottieni_modelli(stato_casuale=STATO_CASUALE)
    modelli_standard_clust = {nome: modello for nome, modello in modelli_standard_clust.items() if nome != 'Naive Bayes'}

    risultati_con_cluster = ottimizza_e_valuta_modelli_cv(
        modelli_standard_clust, griglie_standard, X_proc_con_cluster, y_target,
        num_split_esterni=NUM_SPLIT_ESTERNI, num_split_interni=NUM_SPLIT_INTERNI, stato_casuale=STATO_CASUALE
    )
else:
    print("valutazione modelli normali con cluster saltata")

In [ ]:
#riepilogo risultati
print("riepilogo risultati")
risultati_finali_combinati = {}
if risultati_nb:
    risultati_finali_combinati.update(risultati_nb)
if risultati_no_cluster:
    risultati_finali_combinati.update(risultati_no_cluster)
if risultati_con_cluster:
    for nome_modello, dati_ris in risultati_con_cluster.items():
        nome_nuovo = nome_modello.replace(" (Tuned)", " + Clust (Tuned)") if " (Tuned)" in nome_modello else nome_modello + " + Clust"
        risultati_finali_combinati[nome_nuovo] = dati_ris

if risultati_finali_combinati:
    df_risultati_finali = stampa_risultati_cv(risultati_finali_combinati)
    df_risultati_finali.to_csv('riepilogo_risultati_progetto.csv')
else:
    print("nessun risultato")